In [10]:
# Imports the Google Cloud client library
from google.cloud import storage

# Instantiates a client
storage_client = storage.Client()

# The name for the new bucket
bucket_name = "budfox-data-science-development"

# Creates the new bucket
bucket = storage_client.get_bucket(bucket_name)

print("Bucket {} created.".format(bucket.name))
# Note: Client.list_blobs requires at least package version 1.17.0.
blobs = storage_client.list_blobs(bucket_name)

assets = [blob.name.split("/")[-1].split(".")[0] for blob in blobs]
print(len(assets))
uris = ["gs://budfox-data-science-development/Data/" + asset + ".csv" for asset in assets]
for i, uri in enumerate(uris):
    if i > 10:
        break
    else:
        print(i, uri)

Bucket budfox-data-science-development created.
251
0 gs://budfox-data-science-development/Data/AAVEETH.csv
1 gs://budfox-data-science-development/Data/AAVEEUR.csv
2 gs://budfox-data-science-development/Data/AAVEGBP.csv
3 gs://budfox-data-science-development/Data/AAVEUSD.csv
4 gs://budfox-data-science-development/Data/AAVEXBT.csv
5 gs://budfox-data-science-development/Data/ADAETH.csv
6 gs://budfox-data-science-development/Data/ADAEUR.csv
7 gs://budfox-data-science-development/Data/ADAUSD.csv
8 gs://budfox-data-science-development/Data/ADAUSDT.csv
9 gs://budfox-data-science-development/Data/ADAXBT.csv
10 gs://budfox-data-science-development/Data/ALGOETH.csv


In [13]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set table_id to the ID of the table to create.
# table_id = "your-project.your_dataset.your_table_name"

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("time_stamp", "TIMESTAMP"),
        bigquery.SchemaField("price", "FLOAT"),
        bigquery.SchemaField("volume", "FLOAT")#, 
        #bigquery.SchemaField("asset", "STRING")
    ],
    skip_leading_rows=0,
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
)
for uri, table in zip(uris, assets):
        
    table_id= "development-302415.markets." + table 
    load_job = client.load_table_from_uri(
        uri, table_id, job_config=job_config
    )  # Make an API request.

    load_job.result()  # Waits for the job to complete.

    destination_table = client.get_table(table_id)  # Make an API request.
    print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 1461 rows.
Loaded 6248 rows.
Loaded 368 rows.
Loaded 6088 rows.
Loaded 2379 rows.
Loaded 225581 rows.
Loaded 1612232 rows.
Loaded 1178759 rows.
Loaded 15086 rows.
Loaded 798295 rows.
Loaded 51289 rows.
Loaded 258108 rows.
Loaded 244308 rows.
Loaded 134059 rows.
Loaded 2043 rows.
Loaded 9860 rows.
Loaded 7720 rows.
Loaded 2816 rows.
Loaded 117767 rows.
Loaded 511124 rows.
Loaded 414077 rows.
Loaded 374009 rows.
Loaded 1763 rows.
Loaded 21700 rows.
Loaded 4266 rows.
Loaded 21953 rows.
Loaded 17113 rows.
Loaded 9991 rows.
Loaded 38792 rows.
Loaded 238358 rows.
Loaded 188388 rows.
Loaded 146902 rows.
Loaded 4035 rows.
Loaded 26663 rows.
Loaded 5279798 rows.
Loaded 7509 rows.
Loaded 629 rows.
Loaded 4838190 rows.
Loaded 52831 rows.
Loaded 2964327 rows.
Loaded 45758 rows.
Loaded 117022 rows.
Loaded 119265 rows.
Loaded 66429 rows.
Loaded 11498 rows.
Loaded 109955 rows.
Loaded 87971 rows.
Loaded 32822 rows.
Loaded 206931 rows.
Loaded 244445 rows.
Loaded 96450 rows.
Loaded 1628570 rows.


In [35]:
queries = ["SELECT time_stamp, price, volume, '" + asset + "' as asset FROM `development-302415.markets." + asset + "`" for asset in assets]
for q in queries[:3]:
    print(q)

SELECT time_stamp, price, volume, 'AAVEETH' as asset FROM `development-302415.markets.AAVEETH`
SELECT time_stamp, price, volume, 'AAVEEUR' as asset FROM `development-302415.markets.AAVEEUR`
SELECT time_stamp, price, volume, 'AAVEGBP' as asset FROM `development-302415.markets.AAVEGBP`


In [30]:
%%bigquery results
SELECT time_stamp, price, volume, 'AAVEEUR' as asset FROM development-302415.markets.AAVEEUR

In [31]:
results

,time_stamp,price,volume,asset
0,2020-12-23 22:21:10+00:00,56.00,1.497604,AAVEEUR
1,2020-12-23 22:21:10+00:00,56.00,0.002392,AAVEEUR
2,2020-12-23 22:21:10+00:00,56.00,0.000004,AAVEEUR
3,2020-12-23 22:21:10+00:00,56.00,7.310005,AAVEEUR
4,2020-12-23 22:21:10+00:00,56.00,0.011677,AAVEEUR
...,...,...,...,...
6243,2020-12-18 13:33:17+00:00,69.84,0.100100,AAVEEUR
6244,2020-12-22 02:07:44+00:00,66.59,0.100100,AAVEEUR
6245,2020-12-21 00:37:03+00:00,70.59,0.100100,AAVEEUR
6246,2020-12-18 22:24:36+00:00,71.09,0.100100,AAVEEUR


In [36]:
giant_query = " UNION ALL ".join(queries)

In [37]:
print(giant_query)

SELECT time_stamp, price, volume, 'AAVEETH' as asset FROM `development-302415.markets.AAVEETH` UNION ALL SELECT time_stamp, price, volume, 'AAVEEUR' as asset FROM `development-302415.markets.AAVEEUR` UNION ALL SELECT time_stamp, price, volume, 'AAVEGBP' as asset FROM `development-302415.markets.AAVEGBP` UNION ALL SELECT time_stamp, price, volume, 'AAVEUSD' as asset FROM `development-302415.markets.AAVEUSD` UNION ALL SELECT time_stamp, price, volume, 'AAVEXBT' as asset FROM `development-302415.markets.AAVEXBT` UNION ALL SELECT time_stamp, price, volume, 'ADAETH' as asset FROM `development-302415.markets.ADAETH` UNION ALL SELECT time_stamp, price, volume, 'ADAEUR' as asset FROM `development-302415.markets.ADAEUR` UNION ALL SELECT time_stamp, price, volume, 'ADAUSD' as asset FROM `development-302415.markets.ADAUSD` UNION ALL SELECT time_stamp, price, volume, 'ADAUSDT' as asset FROM `development-302415.markets.ADAUSDT` UNION ALL SELECT time_stamp, price, volume, 'ADAXBT' as asset FROM `dev